In [3]:
!pip install numpy pandas python-binance scikit-learn tensorflow

In [4]:
import numpy as np
import pandas as pd
import time
import os
from binance.client import Client
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Input, Bidirectional, Dropout, Attention
from tensorflow.keras.optimizers import Adam

In [5]:
# Import and Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define Model Save Path in Google Drive
MODEL_PATH = "/content/drive/MyDrive/TradingBot/lstm_trading_model.h5"
print(f"Model will be saved at: {MODEL_PATH}")

Mounted at /content/drive
Model will be saved at: /content/drive/MyDrive/TradingBot/lstm_trading_model.h5


In [ ]:
# Binance Testnet API Credentials (Replace with your own keys from Testnet)
from binance.client import Client

API_KEY = os.getenv("API_KEY")
API_SECRET = os.getenv("API_SECRET")
client = Client(API_KEY, API_SECRET, tld='com', testnet=True)

In [7]:
# Function to fetch historical OHLCV data
def get_historical_data(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1MINUTE, lookback='365 days ago UTC'):
    try:
        klines = client.get_historical_klines(symbol, interval, lookback)
        df = pd.DataFrame(klines, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
                                           'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
                                           'Taker buy quote asset volume', 'Ignore'])
        df = df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)
        return df
    except Exception as e:
        print(f"Error fetching historical data: {e}")
        return None

data = get_historical_data()
if data is None:
    raise RuntimeError("Failed to fetch historical data!")

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

In [8]:
# Function to create time-series sequences
def create_sequences(data, time_steps=60):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps, 3])  # Predicting 'Close' price
    return np.array(X), np.array(y)

TIME_STEPS = 60
X, y = create_sequences(data_scaled, TIME_STEPS)
X_train, X_test, y_train, y_test = X[:int(0.8*len(X))], X[int(0.8*len(X)):], y[:int(0.8*len(y))], y[int(0.8*len(y)):]


In [9]:
# Build Advanced LSTM Model
def build_model():
    inputs = Input(shape=(TIME_STEPS, 5))
    x = Bidirectional(LSTM(128, return_sequences=True))(inputs)
    x = Dropout(0.2)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    attention = Attention()([x, x])
    x = Bidirectional(LSTM(64, return_sequences=False))(attention)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='linear')(x)
    model = tf.keras.Model(inputs, output)
    model.compile(optimizer=Adam(learning_rate=0.001, decay=1e-6), loss='mse')
    return model

In [10]:
# Load or Train Model
if os.path.exists(MODEL_PATH):
    model = load_model(MODEL_PATH)
    print("Model loaded successfully!")
else:
    model = build_model()
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
    model.save(MODEL_PATH)
    print(f"Model trained and saved at: {MODEL_PATH}")

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


722/722 ━━━━━━━━━━━━━━━━━━━━ 25s 25ms/step - loss: 0.0148 - val_loss: 5.4725e-04
Epoch 2/50
722/722 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - loss: 0.0018 - val_loss: 5.7198e-05
Epoch 3/50
722/722 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - loss: 0.0015 - val_loss: 3.3616e-05
Epoch 4/50
722/722 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - loss: 0.0013 - val_loss: 2.5023e-04
Epoch 5/50
722/722 ━━━━━━━━━━━━━━━━━━━━ 17s 23ms/step - loss: 0.0011 - val_loss: 6.9645e-05
Epoch 6/50
722/722 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - loss: 8.9452e-04 - val_loss: 9.9717e-05
Epoch 7/50
722/722 ━━━━━━━━━━━━━━━━━━━━ 22s 23ms/step - loss: 6.7019e-04 - val_loss: 2.8105e-04
Epoch 8/50
722/722 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - loss: 6.0629e-04 - val_loss: 6.7540e-05
Epoch 9/50
722/722 ━━━━━━━━━━━━━━━━━━━━ 21s 23ms/step - loss: 4.1094e-04 - val_loss: 5.6160e-05
Epoch 10/50
722/722 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - loss: 2.9527e-04 - val_loss: 2.8294e-05
Epoch 11/50
722/722 ━━━━━━━━━━━━━━━━━━━━ 20s 23ms/step - loss: 2.8533e

Model trained and saved at: /content/drive/MyDrive/TradingBot/lstm_trading_model.h5


In [11]:
def predict_next_1_minute(symbol='BTCUSDT'):
    try:
        latest_data = get_historical_data(symbol)  # Get last TIME_STEPS values
        latest_scaled = scaler.transform(latest_data)
        latest_scaled = np.expand_dims(latest_scaled, axis=0)

        prediction = model.predict(latest_scaled)  # Predict normalized close price
        predicted_prices = scaler.inverse_transform(
            np.hstack((latest_data[-1, :-1], prediction[0])).reshape(1, -1)
        )[:, -1]  # Extract predicted Close price

        return predicted_prices[0]
    except Exception as e:
        print(f"Error predicting next 1 minute: {e}")
        return None

In [12]:
def predict_next_1_minute(symbol='BTCUSDT'):
    try:
        latest_data = get_historical_data().values[-TIME_STEPS:]
        print(f"Latest Data Shape: {latest_data.shape}")  # Debugging

        if latest_data.shape[0] != TIME_STEPS:
            raise ValueError(f"Expected shape ({TIME_STEPS}, 5), but got {latest_data.shape}")

        latest_scaled = scaler.transform(latest_data)
        latest_scaled = np.expand_dims(latest_scaled, axis=0)
        print(f"Scaled Data Shape: {latest_scaled.shape}")  # Debugging

        prediction = model.predict(latest_scaled)
        print(f"Prediction Shape: {prediction.shape}")  # Debugging

        # Ensure the transformation aligns with training features
        predicted_prices = scaler.inverse_transform(
            np.hstack((np.zeros((1, latest_data.shape[1] - 1)), prediction.reshape(-1, 1)))
        )[:, -1]

        return predicted_prices
    except Exception as e:
        print(f"Error predicting next 1 minute: {e}")
        return None


In [16]:
import time

def execute_trade():
    print("Trade executed.")  # Replace with real trade logic

# Running real-time prediction & trading loop (Simulated)
try:
    while True:
        execute_trade()
        time.sleep(60)  # Wait 1 minute before next execution
except KeyboardInterrupt:
    print("\nTrading bot stopped.")


Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.
Trade executed.

Trading bot stopped.
